## Notebook Start

### Load Packages

In [93]:
import requests
import pandas as pd

In [94]:
api_url = "https://uclabruins.com/api/v2/EventsResults/results?sportId=9&$pageIndex=0&$pageSize=50"
response = requests.get(api_url)


In [95]:
data = response.json()
data["items"][0]

{'id': 34034,
 'gamePregameStoryId': 69064,
 'gamePostgameStoryId': 69072,
 'date': '2025-05-16T17:00:00',
 'endDate': None,
 'dateUtc': '2025-05-17T00:00:00.0000001Z',
 'endDateUtc': None,
 'time': '5:00 PM',
 'isDoubleHeader': False,
 'showAtVs': True,
 'tbd': False,
 'allDay': False,
 'teamPrefix': None,
 'status': 'O',
 'locationIndicator': 'N',
 'neutralHometeam': False,
 'location': 'Waco, Texas',
 'isConference': False,
 'conference': 'Big Ten',
 'conferenceAbbrev': 'Big Ten',
 'conferenceLogo': None,
 'isSpotlight': False,
 'type': 'recent',
 'tournament': "NCAA Men's Tennis Championship",
 'ticketLink': None,
 'sport': {'nonSport': False,
  'atVs': True,
  'gameSynonym': 'Match',
  'globalSportShortName': 'mten',
  'shortTitle': '',
  'scheduleId': 2675,
  'abbrev': 'MTEN',
  'shortDisplay': None,
  'shortName': 'mten',
  'globalSportId': None,
  'globalSportNameSlug': None,
  'globalSportGender': 'm',
  'id': 9,
  'title': "Men's Tennis"},
 'schedule': {'id': 2675,
  'title':

In [96]:
data["items"][0].get("result")

{'status': 'L',
 'teamScore': '1',
 'opponentScore': '4',
 'preScore': None,
 'postScore': None,
 'boxScore': '/boxscore.aspx?id=34034',
 'gameId': 34034,
 'lineScores': {'game_winner': 'H',
  'this_team_is_home_team': False,
  'home_full_name': 'Texas',
  'home_short_name': 'TEX',
  'away_full_name': 'UCLA',
  'away_short_name': 'UCLA',
  'period_label': None,
  'periods': ['F'],
  'period_home_score': ['4'],
  'period_away_score': ['1']}}

In [97]:
bsurllist = []
box_score_api_urls = []

if response.status_code == 200:
    data = response.json()
    all_games = []

    for game in data.get("items", []):
        game_data = {}
        game_data["date"] = game.get("date", "No date available").split("T")[0]
        game_data["location"] = game.get("location", "No location available")
        
        opponent_title = game.get("opponent", {}).get("title", "")
        opponent_parts = opponent_title.split()

        # Only assign seed if it exists
        seed = opponent_parts[1] if len(opponent_parts) > 1 else None
        game_data["opponent_seed"] = seed if seed and seed.isdigit() else None

        game_data["isConference"] = game.get("isConference")
        game_data["tournament"] = game.get("tournament")
        
        result = game.get("result", {}) or {}
        boxScore = result.get("boxScore")
        if isinstance(boxScore, str) and boxScore.startswith("/"):
            boxScore = "https://uclabruins.com/api/v2/Stats/boxscore/" + boxScore.split("=")[-1]
        game_data["boxScore"] = boxScore

        game_data["overallScore"] = str(result.get("teamScore")) + '-' + str(result.get("opponentScore"))

        lineScores = result.get("lineScores", {}) or {}

        winner_code = lineScores.get("game_winner")
        winner_label = {'H': 'home', 'A': 'away'}.get(winner_code, None)

        game_data["game_winner_label"] = winner_label   

        game_data["this_team_is_home_team"] = lineScores.get("this_team_is_home_team")
        game_data["home_full_name"] = lineScores.get("home_full_name")
        game_data["away_full_name"] = lineScores.get("away_full_name")
        
        facility = game.get("gameFacility", {}) or {}
        game_data["facility_id"] = facility.get("id")
        game_data["facility_location"] = facility.get("title")

        all_games.append(game_data)

    df = pd.DataFrame(all_games)
else:
    print(f"Error: Unable to fetch data (Status Code: {response.status_code})")


In [98]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df.head(25)

,date,location,opponent_seed,isConference,tournament,boxScore,overallScore,game_winner_label,this_team_is_home_team,home_full_name,away_full_name,facility_id,facility_location
0,2025-05-16,"Waco, Texas",3,False,NCAA Men's Tennis Championship,https://uclabruins.com/api/v2/Stats/boxscore/3...,1-4,home,False,Texas,UCLA,339.0,Hurd Tennis Center
1,2025-05-09,"Los Angeles, Calif.",None,False,NCAA Super-Regional Round,https://uclabruins.com/api/v2/Stats/boxscore/3...,4-2,home,False,UCLA,USC,10.0,Los Angeles Tennis Center
2,2025-05-03,"Berkeley, Calif.",11,False,NCAA First and Second Rounds,https://uclabruins.com/api/v2/Stats/boxscore/3...,4-2,away,False,California,UCLA,213.0,Hellman Tennis Complex
3,2025-05-02,"Berkeley, Calif.",None,False,NCAA First and Second Rounds,https://uclabruins.com/api/v2/Stats/boxscore/3...,4-0,home,False,UCLA,UC Santa Barbara,213.0,Hellman Tennis Complex
4,2025-04-27,"Columbus, Ohio",1,False,Big Ten Men's Tennis Tournament,https://uclabruins.com/api/v2/Stats/boxscore/3...,4-3,away,False,Ohio State,UCLA,359.0,Auer Tennis Complex
5,2025-04-26,"Columbus, Ohio",3,False,Big Ten Men's Tennis Tournament,https://uclabruins.com/api/v2/Stats/boxscore/3...,4-0,home,False,UCLA,Michigan State,359.0,Auer Tennis Complex
6,2025-04-25,"Columbus, Ohio",7,False,Big Ten Men's Tennis Tournament,https://uclabruins.com/api/v2/Stats/boxscore/3...,4-1,home,False,UCLA,Michigan,211.0,Ty Tucker Tennis Center
7,2025-04-20,"Lincoln, Neb.",41,True,None,https://uclabruins.com/api/v2/Stats/boxscore/3...,4-1,away,False,Nebraska,UCLA,329.0,Sid and Hazel Dillon Tennis Center
8,2025-04-18,"Madison, Wis.",58,True,None,https://uclabruins.com/api/v2/Stats/boxscore/3...,4-0,away,False,Wisconsin,UCLA,328.0,Nielsen Tennis Stadium
9,2025-04-13,"Los Angeles, Calif.",28,True,None,https://uclabruins.com/api/v2/Stats/boxscore/3...,4-1,home,False,UCLA,Michigan State,10.0,Los Angeles Tennis Center


### Parse Individual Match Links

In [99]:
url = df.loc[0, 'boxScore']
url

'https://uclabruins.com/api/v2/Stats/boxscore/34034'

In [100]:
response = requests.get(url, timeout=10) 
response.raise_for_status() 
data = response.json()
data

{'pdfDoc': 'https://dxbhsrqyrr690.cloudfront.net/sidearm.nextgen.sites/uclabruins.com/stats/mten/2024/pdf/20250517121633-250516M.pdf',
 'thisTeamIsHomeTeam': False,
 'venue': {'date': '5/16/2025',
  'oldDate': '0',
  'time': '7:00 PM',
  'start': '7:00 PM',
  'end': '0',
  'duration': '',
  'location': 'Waco, TX',
  'arena': 'Hurd Tennis Center',
  'stadium': None,
  'isALeagueGame': False,
  'isNeutral': True,
  'isAPostseasonGame': True,
  'attendance': '0',
  'officials': [],
  'rules': {'doublesone': '1',
   'doublestwo': '0',
   'indivonly': '0',
   'suspended': '0',
   'format': '1',
   'squash': '0'},
  'notes': ['NCAA Championship - Quarterfinal Round'],
  'tournament': False,
  'tournamentName': ''},
 'homeTeam': {'id': 'TEX',
  'code': '',
  'name': 'Texas',
  'logo': '/images/logos/texas_200x200.png',
  'record': '((29-4))',
  'isTenantTeam': False,
  'players': [{'name': 'Pierre-Yves Bailly',
    'oldName': 'Pierre-Yves Bailly',
    'playerFirstLastName': None,
    'checkNa

In [101]:
singles_matches = data.get("singles", [])
singles_matches

[{'matchNum': '1',
  'order': '2',
  'tournament': None,
  'vh': 'V',
  'team': 'UCLA',
  'teamId': 'UCLA',
  'uniform1': '4',
  'name1': 'Quan, Rudy',
  'rosterId1': '14692',
  'playerUrl1': '/sports/mens-tennis/roster/rudy-quan/14692',
  'uniform2': '0',
  'name2': '0',
  'rosterId2': None,
  'playerUrl2': None,
  'set1': '5',
  'set2': '2',
  'set3': '',
  'set4': '',
  'set5': '',
  'tie1': '',
  'tie2': '',
  'tie3': '',
  'ranknat': '20',
  'rankreg': '',
  'rank': ' (20/-)',
  'isWinner': '0'},
 {'matchNum': '1',
  'order': '2',
  'tournament': None,
  'vh': 'H',
  'team': 'Texas',
  'teamId': 'TEX',
  'uniform1': '7',
  'name1': 'Timo Legout',
  'rosterId1': None,
  'playerUrl1': None,
  'uniform2': '0',
  'name2': '0',
  'rosterId2': None,
  'playerUrl2': None,
  'set1': '7',
  'set2': '6',
  'set3': '',
  'set4': '',
  'set5': '',
  'tie1': '',
  'tie2': '',
  'tie3': '',
  'ranknat': '1',
  'rankreg': '',
  'rank': ' (1/-)',
  'isWinner': '1'},
 {'matchNum': '2',
  'order': 

In [102]:
singles_completed_order = data.get("finishOrderSingles", [])
singles_completed_order

'2, 1, 4, 3'

In [103]:
singles_clinched = int(singles_completed_order[-1] if singles_completed_order else 0)
singles_clinched

3

In [104]:
singles_matches[0].get("matchNum", "").strip()

'1'

In [105]:
list(range(0, len(singles_matches), 2))

[0, 2, 4, 6, 8, 10]

In [106]:
singles_matches[0:2]

[{'matchNum': '1',
  'order': '2',
  'tournament': None,
  'vh': 'V',
  'team': 'UCLA',
  'teamId': 'UCLA',
  'uniform1': '4',
  'name1': 'Quan, Rudy',
  'rosterId1': '14692',
  'playerUrl1': '/sports/mens-tennis/roster/rudy-quan/14692',
  'uniform2': '0',
  'name2': '0',
  'rosterId2': None,
  'playerUrl2': None,
  'set1': '5',
  'set2': '2',
  'set3': '',
  'set4': '',
  'set5': '',
  'tie1': '',
  'tie2': '',
  'tie3': '',
  'ranknat': '20',
  'rankreg': '',
  'rank': ' (20/-)',
  'isWinner': '0'},
 {'matchNum': '1',
  'order': '2',
  'tournament': None,
  'vh': 'H',
  'team': 'Texas',
  'teamId': 'TEX',
  'uniform1': '7',
  'name1': 'Timo Legout',
  'rosterId1': None,
  'playerUrl1': None,
  'uniform2': '0',
  'name2': '0',
  'rosterId2': None,
  'playerUrl2': None,
  'set1': '7',
  'set2': '6',
  'set3': '',
  'set4': '',
  'set5': '',
  'tie1': '',
  'tie2': '',
  'tie3': '',
  'ranknat': '1',
  'rankreg': '',
  'rank': ' (1/-)',
  'isWinner': '1'}]

In [107]:
# Helper Function: Format Name
def format_name(name):

    if isinstance(name, str) and ',' in name:
        return " ".join(s.strip() for s in name.split(",")[::-1])
    return name

In [108]:
# Helper Function to Create Score
def build_tennis_score(match1, match2):

    score_parts = []

    for i in range(1, 4):  # set1 to set3
        set_key = f'set{i}'
        tie_key = f'tie{i}'

        set1_score = match1.get(set_key)
        set2_score = match2.get(set_key)

        # Unfinished matches, don't append/continue function
        if str(set1_score) == '95' or str(set2_score) == '95':
            continue

        # Skip if both scores are missing
        if not set1_score or not set2_score:
            continue

        set_score = f"{set1_score}-{set2_score}"

        # Handle tiebreak
        tie1 = match1.get(tie_key)
        tie2 = match2.get(tie_key)

        if tie1 is not None and tie2 is not None:
            try:
                tie1 = int(tie1)
                tie2 = int(tie2)

                if (set_score == '7-6') or (set_score == '6-7'):
                    # Add the **loser's** tiebreak score in parentheses
                    tiebreak_loser_score = str(min(tie1, tie2))
                    set_score += f"({tiebreak_loser_score})"
            except ValueError:
                pass  # Ignore invalid tiebreak data

        score_parts.append(set_score)

    return ", ".join(score_parts)

In [109]:
date = data.get('gameDate')
home_team_record = data.get('homeTeamRecord').strip("()")
away_team_record = data.get('visitingTeamRecord').strip("()")

In [110]:
# Create empty dictionary
player_pairs = []

for i in range(0, len(singles_matches), 2):
    singles_number = int(singles_matches[i]['matchNum'])
    order_finished = singles_matches[i]['order']
    is_clinched = singles_number == singles_clinched
    player1_school = singles_matches[i]['team']
    player2_school = singles_matches[i+1]['team']
    player1_name = format_name(singles_matches[i]['name1'])
    player2_name = format_name(singles_matches[i+1]['name1'])
    score = build_tennis_score(singles_matches[i], singles_matches[i+1])

    if singles_matches[i]['isWinner'] == '1':
        winner = player1_name
    elif singles_matches[i+1]['isWinner'] == '1':
        winner = player2_name
    elif singles_matches[i]['isWinner'] == '0' and singles_matches[i+1]['isWinner'] == '0':
        winner = 'Unfinished'
    else:
        winner = None



    player_pairs.append({
                        'singles_number': singles_number,
                        'order_finished': order_finished,
                        'clincher': is_clinched,
                        'player1_school': player1_school,
                        'player2_school': player2_school,
                        'player1_name': player1_name, 
                        'player2_name': player2_name,
                        'score': score,
                        'match_winner': winner,
                        'date': date,
                        'home_team_record': home_team_record,
                        'away_team_record': away_team_record
                         })
                         

# Convert to DataFrame
df_pairs = pd.DataFrame(player_pairs)

In [111]:
for url in df['boxScore']:
    print(url)

https://uclabruins.com/api/v2/Stats/boxscore/34034
https://uclabruins.com/api/v2/Stats/boxscore/34033
https://uclabruins.com/api/v2/Stats/boxscore/34217
https://uclabruins.com/api/v2/Stats/boxscore/34032
https://uclabruins.com/api/v2/Stats/boxscore/34211
https://uclabruins.com/api/v2/Stats/boxscore/34210
https://uclabruins.com/api/v2/Stats/boxscore/34031
https://uclabruins.com/api/v2/Stats/boxscore/34030
https://uclabruins.com/api/v2/Stats/boxscore/34029
https://uclabruins.com/api/v2/Stats/boxscore/34028
https://uclabruins.com/api/v2/Stats/boxscore/34027
https://uclabruins.com/api/v2/Stats/boxscore/34026
https://uclabruins.com/api/v2/Stats/boxscore/34025
https://uclabruins.com/api/v2/Stats/boxscore/34020
https://uclabruins.com/api/v2/Stats/boxscore/34024
https://uclabruins.com/api/v2/Stats/boxscore/34023
https://uclabruins.com/api/v2/Stats/boxscore/34022
https://uclabruins.com/api/v2/Stats/boxscore/34021
https://uclabruins.com/api/v2/Stats/boxscore/34019
https://uclabruins.com/api/v2/S


## Completed Function

### Load Packages

In [112]:
import requests
import pandas as pd

##### Helper Functions

In [113]:
# Helper Function: Format Name
def format_name(name):

    if isinstance(name, str) and ',' in name:
        return " ".join(s.strip() for s in name.split(",")[::-1])
    return name

In [114]:
# Helper Function to Create Score
def build_tennis_score(match1, match2):

    score_parts = []

    for i in range(1, 4):  # set1 to set3
        set_key = f'set{i}'
        tie_key = f'tie{i}'

        set1_score = match1.get(set_key)
        set2_score = match2.get(set_key)

        # Unfinished matches, don't append/continue function
        if str(set1_score) == '95' or str(set2_score) == '95':
            continue

        # Skip if both scores are missing
        if not set1_score or not set2_score:
            continue

        set_score = f"{set1_score}-{set2_score}"

        # Handle tiebreak
        tie1 = match1.get(tie_key)
        tie2 = match2.get(tie_key)

        if tie1 is not None and tie2 is not None:
            try:
                tie1 = int(tie1)
                tie2 = int(tie2)

                if (set_score == '7-6') or (set_score == '6-7'):
                    # Add the **loser's** tiebreak score in parentheses
                    tiebreak_loser_score = str(min(tie1, tie2))
                    set_score += f"({tiebreak_loser_score})"
            except ValueError:
                pass  # Ignore invalid tiebreak data

        score_parts.append(set_score)

    return ", ".join(score_parts)

In [119]:
def ucla_bruins(team, year):
    if team.lower() == 'w':
        url = "https://uclabruins.com/api/v2/EventsResults/results?sportId=21&$pageIndex=0&$pageSize=50"
    elif team.lower() == 'm':
        url =  "https://uclabruins.com/api/v2/EventsResults/results?sportId=9&$pageIndex=0&$pageSize=50"

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        all_games = []

        for game in data.get("items", []):
            game_data = {}
            game_data["date"] = game.get("date", "No date available").split("T")[0]
            game_data["location"] = game.get("location", "No location available")
            
            opponent_title = game.get("opponent", {}).get("title", "")
            opponent_parts = opponent_title.split()

            # Only assign seed if it exists
            seed = opponent_parts[1] if len(opponent_parts) > 1 else None
            game_data["opponent_seed"] = seed if seed and seed.isdigit() else None

            game_data["isConference"] = game.get("isConference")
            game_data["tournament"] = game.get("tournament")
            
            result = game.get("result", {}) or {}
            boxScore = result.get("boxScore")
            if isinstance(boxScore, str) and boxScore.startswith("/"):
                boxScore = "https://uclabruins.com/api/v2/Stats/boxscore/" + boxScore.split("=")[-1]
            game_data["boxScore"] = boxScore

            game_data["overallScore"] = str(result.get("teamScore")) + '-' + str(result.get("opponentScore"))

            lineScores = result.get("lineScores", {}) or {}

            winner_code = lineScores.get("game_winner")
            winner_label = {'H': 'home', 'A': 'away'}.get(winner_code, None)

            game_data["game_winner_label"] = winner_label   

            game_data["this_team_is_home_team"] = lineScores.get("this_team_is_home_team")
            game_data["home_full_name"] = lineScores.get("home_full_name")
            game_data["away_full_name"] = lineScores.get("away_full_name")
            
            facility = game.get("gameFacility", {}) or {}
            game_data["facility_id"] = facility.get("id")
            game_data["facility_location"] = facility.get("title")

            all_games.append(game_data)

        df = pd.DataFrame(all_games)
    else:
        print(f"Error: Unable to fetch data (Status Code: {response.status_code})")

    ### Parse each individual URL

    all_matches = []

    for url in df['boxScore'].dropna():
        response = requests.get(url, timeout=10) 
        response.raise_for_status() 
        data = response.json()
        singles_matches = data.get("singles", [])

        date = data.get('gameDate')
        home_team_record = data.get('homeTeamRecord').strip("()")
        away_team_record = data.get('visitingTeamRecord').strip("()")
        
        # Grab singles number clinched
        singles_completed_order = data.get("finishOrderSingles", [])
        singles_clinched = int(singles_completed_order[-1] if singles_completed_order else 0)

        # Create empty dictionary
        player_pairs = []
            
        for i in range(0, len(singles_matches), 2):
            singles_number = int(singles_matches[i]['matchNum'])
            order_finished = singles_matches[i]['order']
            is_clinched = singles_number == singles_clinched
            player1_school = singles_matches[i]['team']
            player2_school = singles_matches[i+1]['team']
            player1_name = format_name(singles_matches[i]['name1'])
            player2_name = format_name(singles_matches[i+1]['name1'])
            score = build_tennis_score(singles_matches[i], singles_matches[i+1])

            if singles_matches[i]['isWinner'] == '1':
                winner = player1_name
            elif singles_matches[i+1]['isWinner'] == '1':
                winner = player2_name
            elif singles_matches[i]['isWinner'] == '0' and singles_matches[i+1]['isWinner'] == '0':
                winner = 'Unfinished'
            else:
                winner = None

            player_pairs.append({
                                'date': date,
                                'singles_number': singles_number,
                                'order_finished': order_finished,
                                'clincher': is_clinched,
                                'player1_school': player1_school,
                                'player2_school': player2_school,
                                'player1_name': player1_name, 
                                'player2_name': player2_name,
                                'score': score,
                                'match_winner': winner,
                                'home_team_record': home_team_record,
                                'away_team_record': away_team_record
                                })
                            
        match_df = pd.DataFrame(player_pairs)
        all_matches.append(match_df)

    df_all = pd.concat(all_matches, ignore_index=True)

    # Choose the date
    df_all['date'] = pd.to_datetime(df_all['date'], errors='coerce')
    output_df = df_all[df_all['date'].dt.year == year]

    # Output Dataframe
    output_df.to_csv(f'../data/mens/matches_{year}.csv', index=False)

    return print(f'Saved matches_{year}.csv')


        

### Output Csv

In [120]:
ucla_bruins('m', 2025)

Saved matches_2025.csv


In [ ]:

# all_matches = []


# for url in df['boxScore'].dropna():
#     response = requests.get(url, timeout=10) 
#     response.raise_for_status() 
#     data = response.json()
#     singles_matches = data.get("singles", [])

#     date = data.get('gameDate')
#     home_team_record = data.get('homeTeamRecord').strip("()")
#     away_team_record = data.get('visitingTeamRecord').strip("()")
    
#     # Grab singles number clinched
#     singles_completed_order = data.get("finishOrderSingles", [])
#     singles_clinched = int(singles_completed_order[-1] if singles_completed_order else 0)

#     # Create empty dictionary
#     player_pairs = []
        
#     for i in range(0, len(singles_matches), 2):
#         singles_number = int(singles_matches[i]['matchNum'])
#         order_finished = singles_matches[i]['order']
#         is_clinched = singles_number == singles_clinched
#         player1_school = singles_matches[i]['team']
#         player2_school = singles_matches[i+1]['team']
#         player1_name = format_name(singles_matches[i]['name1'])
#         player2_name = format_name(singles_matches[i+1]['name1'])
#         score = build_tennis_score(singles_matches[i], singles_matches[i+1])

#         if singles_matches[i]['isWinner'] == '1':
#             winner = player1_name
#         elif singles_matches[i+1]['isWinner'] == '1':
#             winner = player2_name
#         elif singles_matches[i]['isWinner'] == '0' and singles_matches[i+1]['isWinner'] == '0':
#             winner = 'Unfinished'
#         else:
#             winner = None

#         player_pairs.append({
#                             'date': date,
#                             'singles_number': singles_number,
#                             'order_finished': order_finished,
#                             'clincher': is_clinched,
#                             'player1_school': player1_school,
#                             'player2_school': player2_school,
#                             'player1_name': player1_name, 
#                             'player2_name': player2_name,
#                             'score': score,
#                             'match_winner': winner,
#                             'home_team_record': home_team_record,
#                             'away_team_record': away_team_record
#                             })
                         
#     match_df = pd.DataFrame(player_pairs)
#     all_matches.append(match_df)

# df_all = pd.concat(all_matches, ignore_index=True)
# df_all


In [90]:
# df_all = pd.concat(all_matches, ignore_index=True)
# df_all